In [10]:
import os
from dotenv import load_dotenv
import tiktoken
from revChatGPT.V1 import Chatbot

env_path="../src/.env"
load_dotenv(dotenv_path=env_path)

True

## Limits

OpenAI: 50 requests / hour for each account

## Authentication method
### - Access token
https://chat.openai.com/api/auth/session
```json
{
  "access_token": "<access_token>"
}
```

### - Optional configuration:
```json
{
  "conversation_id": "UUID...",
  "parent_id": "UUID...",
  "proxy": "...",
  "model": "gpt-4", // gpt-4-browsing, text-davinci-002-render-sha, gpt-4, gpt-4-plugins
  "plugin_ids" : ["plugin-d1d6eb04-3375-40aa-940a-c2fc57ce0f51"], // Wolfram Alpha example
  "disable_history": true,
}
```
1. Save this as $HOME/.config/revChatGPT/config.json
2. If you are using Windows, you will need to create an environment variable named HOME and set it to your home profile for the script to be able to locate the config.json file.

Plugin IDs can be found [here](https://github.com/acheong08/ChatGPT/blob/main/docs/plugins.json). Remember to set model to gpt-4-plugins if plugins are enabled. Plugins may or may not work if you haven't installed them from the web interface. You can call chatbot.install_plugin(plugin_id=plugin_id) to install any one of them from code. Call chatbot.get_plugins() to get a list of all plugins available.

In [103]:
def num_tokens_from_messages(messages, model:str = "cl100k_base") -> int:
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(messages))
    return num_tokens


def print_tokens(msg:str, model:str="cl100k_base"):
    encoding = tiktoken.encoding_for_model(model)
    tokens_ids = encoding.encode(msg)

    # for token_id in tokens_ids:
    #     token = encoding.decode([token_id])
    #     print(f"Token ID: {token_id}, Token: {token}")
    print(f"\n\n{model}")
    print(f"Total length: {len(tokens_ids)} tokens. Length:{len(msg.split())} words")
    print("|".join([encoding.decode([token_id]) for token_id in tokens_ids]))
    print("|".join([str(token_id) for token_id in tokens_ids]))


# =====================================================
message = "text example на разных языках"

print(num_tokens_from_messages(message, "gpt-3.5-turbo"))
print_tokens(message, "gpt-3.5-turbo")

10


gpt-3.5-turbo
Total length: 10 tokens. Length:5 words
text| example| на| раз|ных| я|з|ы|ка|х
1342|3187|13373|39479|44786|46410|9136|4655|13433|10693


In [47]:
access_token = os.environ.get("CHAT_GPT_ACCESS_TOKEN")
home = os.environ.get("HOME")
# access_token

In [48]:
chat_config = {
    "access_token": access_token,
    "model": "gpt-4",
    "disable_history": True,
}

chatbot = Chatbot(config=chat_config)

Using chat.openai.com backend


In [14]:
prompt = "how many beaches does portugal have?"

response = ""
for data in chatbot.ask(prompt):
    response = data["message"]
print(response)

As an AI, I don't have real-time data. As of my last training cut-off in September 2021, Portugal has over 800 beaches. However, this number might vary depending on how you count them - some might count smaller, separate sections of the same coastline as different beaches, while others might count them as one. To get the most accurate and up-to-date information, please refer to a reliable source or the Portuguese tourism board.


In [49]:
print(chatbot.conversation_id_prev_queue)
print(chatbot.conversation_id)
print(chatbot.conversation_mapping)
print(chatbot.parent_id)

[]
None
{}
None


In [50]:
chatbot.get_conversations()[:1]

[{'id': '137ee605-79d0-4280-b7df-69c80a8b886e',
  'title': 'Эксперименты с LangChain.',
  'create_time': '2023-06-03T18:31:33.657340+00:00',
  'update_time': '2023-06-03T18:52:07+00:00',
  'mapping': None,
  'current_node': None}]

In [61]:
chatbot.get_conversations()[3]

{'id': '207d1b95-c85e-4cf1-b623-5d0391d173db',
 'title': 'Оптимизация',
 'create_time': '2023-05-31T16:55:14.827552+00:00',
 'update_time': '2023-06-03T19:04:37+00:00',
 'mapping': None,
 'current_node': None}

In [54]:
chat_history = chatbot.get_msg_history("207d1b95-c85e-4cf1-b623-5d0391d173db")

In [63]:
chat_history.keys()

dict_keys(['title', 'create_time', 'update_time', 'mapping', 'moderation_results', 'current_node'])

In [68]:
len(chat_history["mapping"])

66

In [106]:
for map_id in chat_history["mapping"].keys():
    message = chat_history["mapping"][map_id].get("message", None)
    if message is not None:
        role = message["author"]["role"]
        metadata = message["author"]["metadata"]
        status = message["status"]
        weight = message["weight"]

        content = message["content"]
        content_type = content["content_type"]
        content_text = content["parts"][0]
        content_text_len = len(content_text)
        content_text_tokenlen = num_tokens_from_messages(content_text, "gpt-3.5-turbo")

        # print(role, metadata, status, weight, len(content_text))
        print(role, content_text_len, content_text_tokenlen, content_text[:10])



system 0 0 
user 371 159 [Инструкци
assistant 2 2 Ок
user 888 371 Мы делаем 
assistant 2 2 Ок
user 568 165 Есть следу
assistant 2 2 Ок
user 8627 2043 Есть следу
assistant 2 2 Ок
user 61 19 Есть следу
assistant 2 2 Ок
user 3506 869 # constrai
assistant 2 2 Ок
user 5117 1172 # constrai
assistant 2 2 Ок
user 4893 1081 # constrai
assistant 16 8 Ок, продол
user 8686 2057 # constrai
assistant 331 151 Опять же, 
user 51 16 Есть следу
assistant 153 64 Извините з
user 45 23 Подскажи п
assistant 590 253 В начале э
user 44 17 Какими был
assistant 356 141 В самом на
user 40 11 что было д
assistant 324 125 Извините з
user 43 24 какое кол-
assistant 220 93 Извините, 
user 30 13 а если поп
assistant 442 193 Извините з
user 33 12 а количест
assistant 312 121 В настоящи
user 62 32 в начале б
assistant 247 113 Я извиняюс
user 38 18 Приведи пе
assistant 496 210 К сожалени
user 60 26 Приведи по
assistant 489 193 К сожалени
user 198 86 Сэм - это 
assistant 829 330 Сэм, как "
user 46 23 понятно. К
assistant

In [108]:
aaa = 'Есть следующий файл: metrics.py\nСодержимое файла:\n"""Основные бизнесовые метрики."""\nimport typing as tp\nimport pyomo.environ as pyo\n\ndef objective_rule_nsv(model: pyo.ConcreteModel) -> tp.Any:\n    return sum(model.inc_nsv[v] * model.using_idx[v] for v in model.combinations)\n\ndef objective_rule_lsv(model: pyo.ConcreteModel) -> tp.Any:\n    return sum(model.inc_lsv[v] * model.using_idx[v] for v in model.combinations)\n\ndef objective_rule_uplift(model: pyo.ConcreteModel) -> tp.Any:\n    return sum(model.inc_uplift[v] * model.using_idx[v] for v in model.combinations)\n'
print_tokens(aaa, "gpt-3.5-turbo")



gpt-3.5-turbo
Total length: 165 tokens. Length:56 words
Е|сть| след|ующ|ий| файл|:| metrics|.py|
|С|одерж|им|ое| файла|:
|"""|О|с|нов|ные| б|из|н|ес|ов|ые| м|ет|ри|ки|."""
|import| typing| as| tp|
|import| py|omo|.environ| as| py|o|

|def| objective|_rule|_n|sv|(model|:| py|o|.Con|crete|Model|)| ->| tp|.Any|:
|   | return| sum|(model|.inc|_n|sv|[v|]| *| model|.|using|_idx|[v|]| for| v| in| model|.com|binations|)

|def| objective|_rule|_l|sv|(model|:| py|o|.Con|crete|Model|)| ->| tp|.Any|:
|   | return| sum|(model|.inc|_l|sv|[v|]| *| model|.|using|_idx|[v|]| for| v| in| model|.com|binations|)

|def| objective|_rule|_up|lift|(model|:| py|o|.Con|crete|Model|)| ->| tp|.Any|:
|   | return| sum|(model|.inc|_up|lift|[v|]| *| model|.|using|_idx|[v|]| for| v| in| model|.com|binations|)

56191|35592|93129|72119|44938|47418|25|17150|7345|198|19871|75346|16494|44867|71873|512|12885|20976|2297|47805|44065|14391|41873|2156|23297|6856|51736|11562|8341|31203|17165|14781|475|20061|439|18593|198|475|4

In [74]:
chat_history["mapping"]["aaa20fc0-aa97-4108-9d25-56fea02d1da6"]

{'id': 'aaa20fc0-aa97-4108-9d25-56fea02d1da6',
 'message': {'id': 'aaa20fc0-aa97-4108-9d25-56fea02d1da6',
  'author': {'role': 'user', 'metadata': {}},
  'create_time': 1685552114.831427,
  'content': {'content_type': 'text',
   'parts': ['[Инструкции] Сейчас я буду загружать контекст задачи. После получения части информации нет необходимости развивать беседу, достаточно отвечать одним словом: "Ок". Если я буду передавать большой файл - я буду писать дробью какую часть файла я передаю. Например 1/4 - первая часть из четырех, 3/5 - третья часть из пяти. Отвечать развернуто можно только при прямом вопросе.']},
  'status': 'finished_successfully',
  'weight': 1.0,
  'metadata': {'timestamp_': 'absolute'},
  'recipient': 'all'},
 'parent': '836da384-0b30-4723-a22e-ad07cfd20a8e',
 'children': ['0cc07a10-b171-4784-b938-e066b733743d']}

In [77]:
chat_history["mapping"]["aaa20fc0-aa97-4108-9d25-56fea02d1da6"]["message"]["author"]["role"]

'user'

In [118]:
msg = '[Инструкции] Сейчас я буду загружать контекст задачи. После получения части информации нет необходимости развивать беседу, достаточно отвечать одним словом: "Ок". Если я буду передавать большой файл - я буду писать дробью какую часть файла я передаю. Например 1/4 - первая часть из четырех, 3/5 - третья часть из пяти. Отвечать развернуто можно только при прямом вопросе.'
print("Rus:",num_tokens_from_messages(msg, "gpt-3.5-turbo"))

msg = '[Instructions] Currently I will load context. Reply with only "Ok" when ready. For large files, I will indicate the part number (e.g. 1/4). Answer directly only for questions'
print("En:", num_tokens_from_messages(msg, "gpt-3.5-turbo"))

msg = 'Loading context. Reply with "Ok" when ready. For large files, I will indicate the part number (e.g. 1/4). Answer directly only for questions'
print("En:", num_tokens_from_messages(msg, "gpt-3.5-turbo"))

msg = 'Reply with "Ok" when ready. For large files, I will indicate the part number (e.g. 1/4). Answer directly only for questions'
print("En:", num_tokens_from_messages(msg, "gpt-3.5-turbo"))

msg = "Say 'Ok' when ready. I'll note large file parts (e.g., 1/4). Only respond fully to questions"
print("En:", num_tokens_from_messages(msg, "gpt-3.5-turbo"))

msg = "Reply 'Ok' when set. For big files, I'll signal part number (like 1/4). Answer questions directly"
print("En:", num_tokens_from_messages(msg, "gpt-3.5-turbo"))


Rus: 159
En: 42
En: 35
En: 32
En: 27
En: 26
